# Numerical Analysis - IMPA 2020
### Professor Dan Marchesin
### Hallison Paz, 1st year Phd student
### Pedro Fonini, 2nd year MSc student

# Modeling epidemics with SIR model

In [ ]:
import os

import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
from ipywidgets import interact, interact_manual

In [ ]:
OUTPUT_DIR = os.path.join('images', 'SIR')

## Newton's method

In [ ]:
class IterationLimitError(Exception):
    """Exception type for too many iterations.
    
    Signals that we have reached the maximum allowed number of
    iterations for an iterative numerical method."""

In [ ]:
def newton(f, df, err, x0, max_iter=10,
           max_iter_action='fail', verbose=False):
    """Newton's method for solving the equation f(x)=0.
    
    x0: initial guess; shape (N,)
    f: callable; vector valued function of x.
        input: shape (N,)
        output: shape (N,)
    df: callable; for each x, returns the jacobian matrix of f at x.
        input: shape (N,)
        output: shape (N, N)
    err: callable; for each x, return the expected error in the
            numerical evaluation of f(x).
        input: shape (N,)
        output: shape (N,), non-negative entries
    max_iter: integer; the maximum allowed number of iterations
    max_iter_action: action to be taken upon reaching the maximum number
            of iterations. 'return' if we should return the best guess
            up to now; or 'fail' if we should not return and raise an
            expection.
    """
    f_value = f(x0)

    for n_iter in range(max_iter):
        x0 = x0 - linalg.solve(df(x0), f_value)
        f_value = f(x0)
        if np.all(np.abs(f_value) <= err(x0)):
            # f(x0) is so small that it could already be zero, it's
            # impossible to know, so we might as well stop here.
            if verbose:
                print(f'success after n_iter={n_iter}')
            return x0

    if max_iter_action == 'fail':
        raise IterationLimitError(
            "Maximum number of iterations reached and still |f|>err:\n"
            f"    max_iter = {max_iter}\n"
            f"    x = {x0}\n"
            f"    f = {f_value} +/- {err(x0)}")

### Testing Newton's method:

In [ ]:
def f(x):
    return x**2 - 10

def df(x):
    return 2*x

eps = np.finfo(float).eps
def err(x):
    # the value calculated is (x²(1+eps) - 2)(1+eps) =
    # = (x²-2) + ((2+eps)x² - 2)eps
    return 2 * np.abs(x**2 + 1) * eps

sqrt10 = newton(f, df, err, x0=10, verbose=True)
print(sqrt10)
print(sqrt10 - np.sqrt(10))

In [ ]:
def err(x):
    return eps

sqrt10 = newton(f, df, err, x0=10, verbose=True)
print(sqrt10)
print(sqrt10 - np.sqrt(10))

In [ ]:
def f(x):
    # gradient of the rosenbrok function
    yx2 = x[1] - x[0]**2
    return [2*(x[0] - 1) - 400*x[0]*yx2,
            200 * yx2]

def df(x):
    # Hessian of rosenbrok
    Rxx = 2 + 400*(3*x[0]**2 - x[1])
    Rxy = -400 * x[0]
    Ryy = 200
    return [[Rxx, Rxy], [Rxy, Ryy]]

def err(x):
    return [4*(1 + x[0] + 400*x[0]*(x[0]**2+x[1])) * eps,
            (3*x[0]**2 + 2*x[1]) * eps]

minimum = newton(f, df, err, x0=[7, 13], verbose=True)
print(minimum)
print(minimum - [1,1])

### Implicit Euler method

(TO DO)

### Apply the method to SIR

(TO DO)

In [ ]:
def SIR(I0=0.0001,
        dt = 1,
        days = 20,
        beta=2.5, 
        gamma=0.8,
        corrector=False):

    n_samples = int(days/dt) + 1
    t = np.linspace(0, days, n_samples)

    S = np.zeros(n_samples)
    I = np.zeros(n_samples)
    R = np.zeros(n_samples)
    # Initial condition
    R[0] = 0
    I[0] = I0
    S[0] = 1 - I0

    # Step equations forward in time
    for n in range(len(t)-1):
        # predictor
        S[n+1] = S[n] - dt*beta*S[n]*I[n]
        I[n+1] = I[n] + dt*beta*S[n]*I[n] - dt*gamma*I[n] 
        R[n+1] = R[n] + dt*gamma*I[n]
        if corrector:
            S[n+1] = S[n] - dt*beta*S[n+1]*I[n+1]
            I[n+1] = I[n] + dt*beta*S[n+1]*I[n+1] - dt*gamma*I[n+1] 
            R[n+1] = R[n] + dt*gamma*I[n+1]
    # R0 = beta*S[0]/gamma
    # print('R0', R0)
    return S, I, R

In [ ]:
# Initial condition (S0 and R0 are determided by I0)
I0 = 0.1
# Parameters
dt = 1
days = 20
beta = 2.5
gamma = 0.8 
# simulations = {}
S, I, R = SIR(I0, dt, days, beta, gamma)

## Plotting Curves for multiple values of Beta and Gamma

In [ ]:
@interact_manual(I0=(0.0, 1.0, 0.01), 
                 dt=(0.1, 2))
def draw_SIR_curves(I0 = 0.11,
                    dt = 1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8,
                    corrector = False):
    S, I, R = SIR(I0, dt, days, beta, gamma, corrector)
    t = [dt*i for i in range(len(S))]
    fig = plt.figure(figsize=(12,8))
    l1, l2, l3 = plt.plot(t, S, t, I, t, R)
    fig.legend((l1, l2, l3), ('S', 'I', 'R'), 'center right')
    plt.xlabel('Time')
    plt.ylabel('% population')
    plt.show()

## Ploting orbits in the triangle S + I <= 1 (Autonomous system of ODE)

In [ ]:
@interact_manual(samples=(2, 50, 1), 
                 dt=(0.1, 2),
                days=20)
def draw_SI_relation(samples = 11,
                    dt = 0.1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8,
                    corrector = False):
    s_list = []
    i_list = []
    for i0 in np.linspace(0, 1.0, samples):
        S, I, R = SIR(i0, dt, days, beta, gamma, corrector)
        s_list.append(S)
        i_list.append(I)
        

    fig = plt.figure(figsize=(10, 10))
    plt.plot([0.0, 1.0], [1.0, 0.0], 'gray')
    for k in range(len(s_list)):
        plt.plot(s_list[k], i_list[k], 'blue' if k%2 else '#0000ff80')
        
    for i0 in np.linspace(0.0, 0.1, int(samples/2)):
        S, I, R = SIR(i0, dt, days, beta, gamma, corrector)
        plt.plot(S, I, 'red' if k%2 else '#ff000080')
        
    plt.xlabel('Susceptible')
    plt.ylabel('Infected')
    plt.show()

## Comparing Euler Method with and without Predictor-Corrector technique

In [ ]:
@interact_manual(I0=(0.0, 1.0, 0.01), 
                 dt=(0.1, 2),
                 show=['all', 'S', 'I', 'R'])
def compare_methods(I0 = 0.11,
                    dt = 1,
                    days = 20,
                    beta = 2.5,
                    gamma = 0.8,
                    show='all'):
    
    fe_sir = SIR(I0, dt, days, beta, gamma, False)
    pc_sir = SIR(I0, dt, days, beta, gamma, True)
    t =  [dt*i for i in range(len(fe_sir[0]))]
    legends =['S', 'I', 'R']
    
    fig, ax = plt.subplots(1, 3, figsize=(28,8)) if show == 'all' else (plt.figure(figsize=(16,8)), None)
    if show == 'all':
        for index in range(len(fe_sir)):
            ax[index].plot(t, fe_sir[index], label='{} FE'.format(legends[index]))
            ax[index].plot(t, pc_sir[index], label='{} P-C'.format(legends[index]))
            ax[index].legend(loc= 'center right')
    else:
        index = legends.index(show)
        plt.plot(t, fe_sir[index], label='{} FE'.format(legends[index]))
        plt.plot(t, pc_sir[index], label='{} P-C'.format(legends[index]))
        fig.legend(loc= 'center right')
        
    fig.suptitle('Comparinson between methods for ODE')
    plt.xlabel('Time')
    plt.ylabel('Population')
    plt.show()
    